In [ ]:
import pandas as pd
from DataClasses import BloodData
from PredictiveAnalysis import run_ML_analysis
from ResultsUtils import summarize_blood_results
import ConfigDataset as cfg 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


Routine blood and tumor markers analysis

Prediction of current progression:

In [ ]:
SEEDS=range(100, 200)

params=cfg.dataset_params
params.update({
    'is_predictive': True,
    'pred_future_timepoint': False,
    'blood_data_type': 'ROUTINE_AND_TUMOR_MARKERS',
     'data_folder_path': r'./Data',
    'output_exp_path': r'./Results/routine_and_tumor_markers/current_progression_analysis'
    })
blood_data= BloodData(**params)


In [ ]:
blood_data.load_data()
blood_data.clean_data()
blood_data.select_included_exams()
blood_data.drop_unrequired_columns()
blood_data.targets

In [ ]:
blood_data.calc_patients_stats()

Training ML models for current progression prediction

In [ ]:
run_ML_analysis(blood_data_instance=blood_data, seeds=SEEDS, enable_shap=True, models=['RF'])

Results

In [ ]:
_=summarize_blood_results(blood_data.output_exp_path, models=['RF'], metric_col=['roc_auc', 'sensit','specif', 'ppv', 'npv'])

Prediction of future progression (PFS) and prediction of overall survival (OS) using PFS models

In [ ]:
blood_data.pred_future_timepoint= True
blood_data.os_from_pfs= True
blood_data.set_targets()
blood_data.output_exp_path=blood_data.output_exp_path.replace('current_progression_analysis','future_pfs_analysis')
blood_data.targets

In [ ]:
blood_data.select_included_exams()
blood_data.drop_unrequired_columns()
blood_data.selected_df.columns
blood_data.calc_patients_stats()

Training ML models for future progression prediction

In [ ]:
run_ML_analysis(blood_data_instance=blood_data, seeds=SEEDS, enable_shap=True)

Results

In [ ]:
_=summarize_blood_results(blood_data.output_exp_path, models=['RF'], metric_col=['roc_auc', 'sensit','specif', 'ppv', 'npv'])

OS from PFS models results

In [ ]:
os_output_exp_path = f"{blood_data.output_exp_path}_OS_from_PFS"
_=summarize_blood_results(os_output_exp_path)

Sub-analysis:
Prediction of future progression for patients not experiencing progression due to death

In [ ]:
blood_data.os_from_pfs= True
blood_data.output_exp_path=blood_data.output_exp_path.replace('future_pfs_analysis', 'pfs_excluded_prog_due_to_death')
blood_data.exclude_progressed_due_to_death_patients()
blood_data.calc_patients_stats()

Train the models

In [ ]:
run_ML_analysis(blood_data_instance=blood_data, seeds=SEEDS, enable_shap=True)

Results

In [ ]:
_=summarize_blood_results(blood_data.output_exp_path)

OS from PFS

In [ ]:
os_output_exp_path = f"{blood_data.output_exp_path}_OS_from_PFS"
_=summarize_blood_results(os_output_exp_path)